In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from pandas import Series
from datetime import datetime
import numpy as np


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
flights = pd.read_csv('departed_predictive_clean.csv')

C:\Users\MeghanRoffler\AppData\Local\Temp\ipykernel_7616\963386781.py:1: DtypeWarning: Columns (28,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  flights = pd.read_csv('departed_predictive_clean.csv')


In [3]:
flights.head()
# Departure_delay is what we are tryin to predict 

,year,month,day_of_month,day_of_week,flight_date,airline_code,common_airline_code,origin,city_name,state_abbr,state_name,dest,dest_city_name,dest_state_abr,dest_state_name,scheduled_departure_time,departure_delay,dep_time_blk,scheduled_arrival_time,arr_time_blk,scheduled_elapsed_time,distance,distance_group,weather_delay,origin_takeoff_weather Type,origin_takeoff_weather Severity,origin_takeoff_weather StartTime(UTC),origin_takeoff_weather Precipitation(in),dest_takeoff_weather Type,dest_takeoff_weather Severity,dest_takeoff_weather StartTime(UTC),dest_takeoff_weather Precipitation(in)
0,2021,4,1,4,4/1/2021,UA,OO,DEN,"Denver, CO",CO,Colorado,ATY,"Watertown, SD",SD,South Dakota,2021-04-01T17:05:00.000Z,4.0,1000-1059,2021-04-01T18:47:00.000Z,1200-1259,102.0,519.0,3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021,4,2,5,4/2/2021,UA,OO,DEN,"Denver, CO",CO,Colorado,ATY,"Watertown, SD",SD,South Dakota,2021-04-02T17:05:00.000Z,-6.0,1000-1059,2021-04-02T18:47:00.000Z,1200-1259,102.0,519.0,3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021,4,3,6,4/3/2021,UA,OO,DEN,"Denver, CO",CO,Colorado,ATY,"Watertown, SD",SD,South Dakota,2021-04-03T19:05:00.000Z,-8.0,1200-1259,2021-04-03T20:47:00.000Z,1400-1459,102.0,519.0,3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021,4,4,7,4/4/2021,UA,OO,DEN,"Denver, CO",CO,Colorado,ATY,"Watertown, SD",SD,South Dakota,2021-04-04T17:05:00.000Z,-6.0,1000-1059,2021-04-04T18:47:00.000Z,1200-1259,102.0,519.0,3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021,4,5,1,4/5/2021,UA,OO,DEN,"Denver, CO",CO,Colorado,ATY,"Watertown, SD",SD,South Dakota,2021-04-05T17:05:00.000Z,-10.0,1000-1059,2021-04-05T18:47:00.000Z,1200-1259,102.0,519.0,3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# ------------------------------------------ Initial Cleaning ------------------------------------------

In [4]:
# changing columns to datetimes 

flights["scheduled_arrival_time"] = pd.to_datetime(flights["scheduled_arrival_time"])
flights["scheduled_departure_time"] = pd.to_datetime(flights["scheduled_departure_time"])
flights["origin_takeoff_weather StartTime(UTC)"] = pd.to_datetime(flights["origin_takeoff_weather StartTime(UTC)"])
flights["dest_takeoff_weather StartTime(UTC)"] = pd.to_datetime(flights["dest_takeoff_weather StartTime(UTC)"])
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6190358 entries, 0 to 6190357
Data columns (total 32 columns):
 #   Column                                    Dtype              
---  ------                                    -----              
 0   year                                      int64              
 1   month                                     int64              
 2   day_of_month                              int64              
 3   day_of_week                               int64              
 4   flight_date                               object             
 5   airline_code                              object             
 6   common_airline_code                       object             
 7   origin                                    object             
 8   city_name                                 object             
 9   state_abbr                                object             
 10  state_name                                object             
 11  dest       

In [5]:
# creating new calculated columns 

flights["dep_weatherstart_minutes"] = flights["scheduled_departure_time"] - flights["origin_takeoff_weather StartTime(UTC)"]
flights["arr_weatherstart_minutes"] = flights["scheduled_arrival_time"] - flights["dest_takeoff_weather StartTime(UTC)"]

In [6]:
# converting columns from timedelta to minutes 

flights["dep_weatherstart_minutes"] = flights["dep_weatherstart_minutes"] / np.timedelta64(1, 'm')
flights["arr_weatherstart_minutes"] = flights["arr_weatherstart_minutes"] / np.timedelta64(1, 'm')


In [7]:
## dealing with nulls 

# replacing nan weather with "none"
flights["origin_takeoff_weather Type"].replace(np.nan, 'None', inplace = True)
flights["dest_takeoff_weather Type"].replace(np.nan, 'None', inplace = True)

# replacing nan weather severity with "none"
flights["origin_takeoff_weather Severity"].replace(np.nan, 'None', inplace = True)
flights["dest_takeoff_weather Severity"].replace(np.nan, 'None', inplace = True)

In [8]:
## deleting columns 

flights = flights.drop(columns = ['year', 'flight_date', 'day_of_month', 'city_name', 'state_abbr', 'state_name', 'dest_city_name', 'dest_state_abr', 'dest_state_name', 'scheduled_departure_time', 'scheduled_arrival_time'])

In [9]:
## restructuring time block columns

conditions_dep = [
    ((flights['dep_time_blk'] == '0600-0659') | (flights['dep_time_blk'] == '0700-0759') | (flights['dep_time_blk'] == '0800-0859') | (flights['dep_time_blk'] == '0900-0959') | (flights['dep_time_blk'] == '1000-1059') | (flights['dep_time_blk'] == '1100-1159')),
    ((flights['dep_time_blk'] == '1200-1259') | (flights['dep_time_blk'] == '1300-1359') | (flights['dep_time_blk'] == '1400-1459') | (flights['dep_time_blk'] == '1500-1559') | (flights['dep_time_blk'] == '1600-1659') | (flights['dep_time_blk'] == '1700-1759')),
    ((flights['dep_time_blk'] == '1800-1859') | (flights['dep_time_blk'] == '1900-1959') | (flights['dep_time_blk'] == '2000-2059') | (flights['dep_time_blk'] == '2100-2159')),
    ((flights['dep_time_blk'] == '2200-2259') | (flights['dep_time_blk'] == '2300-2359') | (flights['dep_time_blk'] == '0001-0559'))
    ]

conditions_arr = [
    ((flights['arr_time_blk'] == '0600-0659') | (flights['arr_time_blk'] == '0700-0759') | (flights['arr_time_blk'] == '0800-0859') | (flights['arr_time_blk'] == '0900-0959') | (flights['arr_time_blk'] == '1000-1059') | (flights['arr_time_blk'] == '1100-1159')),
    ((flights['arr_time_blk'] == '1200-1259') | (flights['arr_time_blk'] == '1300-1359') | (flights['arr_time_blk'] == '1400-1459') | (flights['arr_time_blk'] == '1500-1559') | (flights['arr_time_blk'] == '1600-1659') | (flights['arr_time_blk'] == '1700-1759')),
    ((flights['arr_time_blk'] == '1800-1859') | (flights['arr_time_blk'] == '1900-1959') | (flights['arr_time_blk'] == '2000-2059') | (flights['arr_time_blk'] == '2100-2159')),
    ((flights['arr_time_blk'] == '2200-2259') | (flights['arr_time_blk'] == '2300-2359') | (flights['arr_time_blk'] == '0001-0559'))
    ]


# create a list of the values we want to assign for each condition
values = ['morning', 'afternoon', 'evening', 'night']

# create a new column and use np.select to assign values to it using our lists as arguments
flights['dep_time'] = np.select(conditions_dep, values)
flights['arr_time'] = np.select(conditions_arr, values)

# remove old time blk colimns 

flights = flights.drop(columns = ['arr_time_blk', 'dep_time_blk'])


In [10]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6190358 entries, 0 to 6190357
Data columns (total 23 columns):
 #   Column                                    Dtype              
---  ------                                    -----              
 0   month                                     int64              
 1   day_of_week                               int64              
 2   airline_code                              object             
 3   common_airline_code                       object             
 4   origin                                    object             
 5   dest                                      object             
 6   departure_delay                           float64            
 7   scheduled_elapsed_time                    float64            
 8   distance                                  float64            
 9   distance_group                            int64              
 10  weather_delay                             float64            
 11  origin_take

# ---------------------------------------- Creating 4 Datasets ----------------------------------------

In [11]:
## set 1: 
# dest_weather: none 
# origin_weather: none 

set1 = flights[(flights["origin_takeoff_weather Type"] == "None") & (flights["dest_takeoff_weather Type"] == "None")]
set1.shape

(5299148, 23)

In [12]:
## set 2: 
# dest_weather: none
# origin_weather: yes 

set2 = flights[(flights["origin_takeoff_weather Type"] != "None") & (flights["dest_takeoff_weather Type"] == "None")]
set2.shape

(419273, 23)

In [13]:
## set 3: 
# dest_weather: yes 
# origin_weather: none 

set3 = flights[(flights["origin_takeoff_weather Type"] == "None") & (flights["dest_takeoff_weather Type"] != "None")]
set3.shape

(408312, 23)

In [14]:
## set 4: 
# dest_weather: yes
# origin_weather: yes 

set4 = flights[(flights["origin_takeoff_weather Type"] != "None") & (flights["dest_takeoff_weather Type"] != "None")]
set4.shape

(63625, 23)

# ------------------------------------------ Set 1 ------------------------------------------

In [15]:
## TESTING OUT - USING DISTANCE GROUP AND AIRLINE_CODE (PLAY AROUND WITH USING DISTANCE AND COMMON_AIRLINE_CODE INSTEAD)
set1 = set1.drop(columns=['common_airline_code', 'distance'])


# dropping columns 
set1 = set1.drop(columns=['origin_takeoff_weather Type', 'origin_takeoff_weather Severity', 'origin_takeoff_weather StartTime(UTC)', 'origin_takeoff_weather Precipitation(in)', 'dest_takeoff_weather Type', 'dest_takeoff_weather Severity', 'dest_takeoff_weather StartTime(UTC)', 'dest_takeoff_weather Precipitation(in)', 'dep_weatherstart_minutes', 'arr_weatherstart_minutes'])
set1.head()

,month,day_of_week,airline_code,origin,dest,departure_delay,scheduled_elapsed_time,distance_group,weather_delay,dep_time,arr_time
0,4,4,UA,DEN,ATY,4.0,102.0,3,0.0,morning,afternoon
1,4,5,UA,DEN,ATY,-6.0,102.0,3,0.0,morning,afternoon
2,4,6,UA,DEN,ATY,-8.0,102.0,3,0.0,afternoon,afternoon
3,4,7,UA,DEN,ATY,-6.0,102.0,3,0.0,morning,afternoon
4,4,1,UA,DEN,ATY,-10.0,102.0,3,0.0,morning,afternoon


In [16]:
# windsorizing departure delay column  (a common range for winsorization is the mean +/- 3 standard deviations)
maxdelay = (set1["departure_delay"].mean()) + (3 * (set1["departure_delay"].std()))
mindelay = (set1["departure_delay"].mean()) - (3 * (set1["departure_delay"].std()))

print(maxdelay)
print(mindelay)

set1["departure_delay"].describe()

# replace dep_delay outliers with maxdelay value 
set1["departure_delay"] = np.where(set1["departure_delay"] > maxdelay, maxdelay, set1["departure_delay"])

146.37858347288346
-129.28203248015785


In [17]:
## creating dummy variables

dummy_set1 = set1.copy()

dummy_set1 = pd.get_dummies(dummy_set1, columns = ['month','day_of_week', 'airline_code', 'origin', 'dest', 'dep_time', 'arr_time'], drop_first=True)
dummy_set1.head()



,departure_delay,scheduled_elapsed_time,distance_group,weather_delay,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5,day_of_week_6,day_of_week_7,airline_code_AS,airline_code_B6,airline_code_DL,airline_code_F9,airline_code_G4,airline_code_HA,airline_code_NK,airline_code_UA,airline_code_WN,origin_ABI,origin_ABQ,origin_ABR,origin_ABY,origin_ACK,origin_ACT,origin_ACV,origin_ACY,origin_ADK,origin_ADQ,origin_AEX,origin_AGS,origin_AKN,origin_ALB,origin_ALO,origin_ALS,origin_ALW,origin_AMA,origin_ANC,origin_APN,origin_ART,origin_ASE,origin_ATL,origin_ATW,origin_ATY,origin_AUS,origin_AVL,origin_AVP,origin_AZA,origin_AZO,origin_BDL,origin_BET,origin_BFF,origin_BFL,origin_BGM,origin_BGR,origin_BHM,origin_BIH,origin_BIL,origin_BIS,origin_BJI,origin_BKG,origin_BLI,origin_BLV,origin_BMI,origin_BNA,origin_BOI,origin_BOS,origin_BPT,origin_BQK,origin_BQN,origin_BRD,origin_BRO,origin_BRW,origin_BTM,origin_BTR,origin_BTV,origin_BUF,origin_BUR,origin_BWI,origin_BZN,origin_CAE,origin_CAK,origin_CDB,origin_CDC,origin_CDV,origin_CGI,origin_CHA,origin_CHO,origin_CHS,origin_CID,origin_CIU,origin_CKB,origin_CLE,origin_CLL,origin_CLT,origin_CMH,origin_CMI,origin_CMX,origin_CNY,origin_COD,origin_COS,origin_COU,origin_CPR,origin_CRP,origin_CRW,origin_CSG,origin_CVG,origin_CWA,origin_CYS,origin_DAB,origin_DAL,origin_DAY,origin_DBQ,origin_DCA,origin_DDC,origin_DEC,origin_DEN,origin_DFW,origin_DHN,origin_DIK,origin_DLG,origin_DLH,origin_DRO,origin_DRT,origin_DSM,origin_DTW,origin_DVL,origin_EAR,origin_EAT,origin_EAU,origin_ECP,origin_EGE,origin_EKO,origin_ELM,origin_ELP,origin_ERI,origin_ESC,origin_EUG,origin_EVV,origin_EWN,origin_EWR,origin_EYW,origin_FAI,origin_FAR,origin_FAT,origin_FAY,origin_FCA,origin_FLG,origin_FLL,origin_FLO,origin_FNT,origin_FOD,origin_FSD,origin_FSM,origin_FWA,origin_GCC,origin_GCK,origin_GEG,origin_GFK,origin_GGG,origin_GJT,origin_GNV,origin_GPT,origin_GRB,origin_GRI,origin_GRK,origin_GRR,origin_GSO,origin_GSP,origin_GST,origin_GTF,origin_GTR,origin_GUC,origin_GUM,origin_HDN,origin_HGR,origin_HHH,origin_HIB,origin_HLN,origin_HNL,origin_HOB,origin_HOU,origin_HPN,origin_HRL,origin_HSV,origin_HTS,origin_HVN,origin_HYA,origin_HYS,origin_IAD,origin_IAG,origin_IAH,origin_ICT,origin_IDA,origin_ILG,origin_ILM,origin_IMT,origin_IND,origin_INL,origin_IPT,origin_ISP,origin_ITH,origin_ITO,origin_JAC,origin_JAN,origin_JAX,origin_JFK,origin_JLN,origin_JMS,origin_JNU,origin_JST,origin_KOA,origin_KTN,origin_LAN,origin_LAR,origin_LAS,origin_LAW,origin_LAX,origin_LBB,origin_LBE,origin_LBF,origin_LBL,origin_LCH,origin_LCK,origin_LEX,origin_LFT,origin_LGA,origin_LGB,origin_LIH,origin_LIT,origin_LNK,origin_LNY,origin_LRD,origin_LSE,origin_LWB,origin_LWS,origin_LYH,origin_MAF,origin_MBS,...,dest_FWA,dest_GCC,dest_GCK,dest_GEG,dest_GFK,dest_GGG,dest_GJT,dest_GNV,dest_GPT,dest_GRB,dest_GRI,dest_GRK,dest_GRR,dest_GSO,dest_GSP,dest_GST,dest_GTF,dest_GTR,dest_GUC,dest_GUM,dest_HDN,dest_HGR,dest_HHH,dest_HIB,dest_HLN,dest_HNL,dest_HOB,dest_HOU,dest_HPN,dest_HRL,dest_HSV,dest_HTS,dest_HVN,dest_HYA,dest_HYS,dest_IAD,dest_IAG,dest_IAH,dest_ICT,dest_IDA,dest_ILG,dest_ILM,dest_IMT,dest_IND,dest_INL,dest_IPT,dest_ISP,dest_ITH,dest_ITO,dest_JAC,dest_JAN,dest_JAX,dest_JFK,dest_JLN,dest_JMS,dest_JNU,dest_JST,dest_KOA,dest_KTN,dest_LAN,dest_LAR,dest_LAS,dest_LAW,dest_LAX,dest_LBB,dest_LBE,dest_LBF,dest_LBL,dest_LCH,dest_LCK,dest_LEX,dest_LFT,dest_LGA,dest_LGB,dest_LIH,dest_LIT,dest_LNK,dest_LNY,dest_LRD,dest_LSE,dest_LWB,dest_LWS,dest_LYH,dest_MAF,dest_MBS,dest_MCI,dest_MCO,dest_MCW,dest_MDT,dest_MDW,dest_MEI,dest_MEM,dest_MFE,dest_MFR,dest_MGM,dest_MHK,dest_MHT,dest_MIA,dest_MKE,dest_MKG,dest_MKK,dest_MLB,dest_MLI,dest_MLU,dest_MOB,dest_MOT,dest_MQT,dest_MRY,dest_MSN,dest_MSO,dest_MSP,dest_MSY,dest_MTJ,dest_MVY,dest_MYR,dest_OAJ,dest_OAK,dest_OGD,dest_OGG,dest_OGS,dest_OKC,dest_OMA,dest_OME,dest_ONT,dest_ORD,dest_ORF,dest_ORH,dest_OTH,dest_OTZ,dest_OWB,dest_PAE,dest_PAH,

In [18]:
dummy_set1.shape

(5299148, 794)

### Set 1 Modeling

In [19]:
# Creating X and y
# y = dummy_set1.pop('departure_delay')
# X = dummy_set1

In [33]:
# subset test
dummy_set1_small = dummy_set1.sample(n = 10000, random_state= 2)

y = dummy_set1_small.pop('departure_delay')
X = dummy_set1_small

In [34]:
X_train, X_test, y_train, y_test = train_test_split(
    X,y, test_size=0.25, random_state=12
)

In [35]:
reg = LinearRegression().fit(X_train, y_train)

In [37]:
reg.score(X_test,y_test)

-48501112872709.01

In [38]:
X.shape

(10000, 793)

In [39]:
### Trying new model 

from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np
n_samples, n_features = 10000, 793
rng = np.random.RandomState(0)
y = rng.randn(n_samples)
X = rng.randn(n_samples, n_features)
regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
regr.fit(X, y)
regr.score(X_test, y_test)

C:\Users\MeghanRoffler\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


-0.055609235130414714

# ------------------------------------------ Notes Space ------------------------------------------

In [23]:
flights["origin_takeoff_weather Severity"].unique()

array(['None', 'Light', 'Severe', 'Moderate', 'UNK', 'Heavy', 'Other'],
      dtype=object)

In [24]:
flights["origin_takeoff_weather Type"].value_counts()

None             5707460
Rain              296980
Fog               120957
Snow               51464
Precipitation       9103
Cold                2952
Storm               1376
Hail                  66
Name: origin_takeoff_weather Type, dtype: int64

In [25]:
## are there weather events with no severity? 


In [26]:
#checks

flights[flights["origin_takeoff_weather Type"] == "Precipitation"]["origin_takeoff_weather Severity"].unique()

flights[flights["origin_takeoff_weather Severity"] == "UNK"]["origin_takeoff_weather Type"].unique()

array(['Precipitation'], dtype=object)

In [27]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6190358 entries, 0 to 6190357
Data columns (total 23 columns):
 #   Column                                    Dtype              
---  ------                                    -----              
 0   month                                     int64              
 1   day_of_week                               int64              
 2   airline_code                              object             
 3   common_airline_code                       object             
 4   origin                                    object             
 5   dest                                      object             
 6   departure_delay                           float64            
 7   scheduled_elapsed_time                    float64            
 8   distance                                  float64            
 9   distance_group                            int64              
 10  weather_delay                             float64            
 11  origin_take